In [1]:
import gspread
import os
import sys
import itertools

import numpy as np
import pandas as pd

In [2]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet_train = gc.open_by_key('1X-P5-djW3Q2hcm_G01zkouCzDseuKLwTHXNX0dpNWdI').sheet1
rows_train = worksheet_train.get_all_values()
df_train = pd.DataFrame(rows_train[1:], columns=rows_train[0])
# handle bool TRUE/FALSE to 1/0
df_train['is_recommended'] = df_train['is_recommended'].map({'TRUE': 1, 'FALSE': 0})
data_train = list(map(tuple, df_train.itertuples(index=False, name=None)))

print(f"Size of Data Train: {len(data_train)}")

print(f"Sample of Data Train: {data_train[:3]}")

Size of Data Train: 1007681
Sample of Data Train: [('136433', '6060', 1), ('2189692', '233980', 1), ('2387112', '204360', 1)]


In [3]:
!pip install --quiet cornac==2.3.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 6.6 MB/s eta 0:00:00


In [4]:
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.1/cu121/repo.html

Looking in links: https://data.dgl.ai/wheels/torch-2.1/cu121/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.3/483.3 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.

In [5]:
import cornac
import dgl

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")

SEED = 42
VERBOSE = True

test_size = 0.2

# for metrics, k = The number of items in the top@k list
k_top = 10

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
System version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Cornac version: 2.3.2


In [6]:
for i in range(len(data_train)):
  data_train[i] = (int(data_train[i][0]), int(data_train[i][1]), int(data_train[i][2]))

In [7]:
# worksheet_output = gc.open_by_key('1gqq8Xm3RGXgcuuUBKrW2_IgXLtKPpSeEsot1TuIb0U4').sheet1

# headers = worksheet_output.row_values(1)

In [8]:
list_results_total = []

# Predictions will be evaluated in terms of ranking, NCRR@50, NDCG@50 and Recall@50.
# k = The number of items in the top@k list
eval_metrics = [
  cornac.metrics.NDCG(k=k_top),
  cornac.metrics.NCRR(k=k_top),
  cornac.metrics.Recall(k=k_top),
]

rs = cornac.eval_methods.RatioSplit(data_train, test_size=test_size, seed=SEED, verbose=VERBOSE)

rating_threshold = 1.0
exclude_unknowns = True


/usr/local/lib/python3.11/dist-packages/cornac/data/dataset.py:335: UserWarning: 2 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Training data:
Number of users = 389629
Number of items = 34638
Number of ratings = 806142
Max rating = 1.0
Min rating = 0.0
Global mean = 0.8
---
Test data:
Number of users = 389629
Number of items = 34638
Number of ratings = 130563
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 389629
Total items = 34638


In [9]:
# Define models
model = cornac.models.EASE(lamb=650, posB=True, verbose=VERBOSE, seed=SEED, name=f"EASE")

In [11]:
# Train and evaluate the model using the RatioSplit evaluation method
cornac.Experiment(eval_method=rs, models=[model], metrics=eval_metrics, user_based=True).run()


[EASE] Training started!

[EASE] Evaluation started!


Ranking:   0%|          | 0/66898 [00:00<?, ?it/s]


TEST:
...
     | NCRR@10 | NDCG@10 | Recall@10 | Train (s) | Test (s)
---- + ------- + ------- + --------- + --------- + --------
EASE |  0.0136 |  0.0180 |    0.0319 |  727.5554 |  99.0640



In [15]:
import requests
from bs4 import BeautifulSoup

def get_steam_game_title_from_id(game_id):
    try:
        url = f"https://store.steampowered.com/app/{game_id}"
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        game_title = soup.find('div', id='appHubAppName')
        if game_title:
            game_title = soup.find('div', id='appHubAppName').text.strip()
        else:
            game_title = ""
        return game_title
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return ""

In [17]:
# list of test user ids
test_user_ids = [1296292, 7660555, 13215940, 13978957, 6984186]

for user_id in test_user_ids:

  recommendations = model.recommend(user_id, k=k_top)

  print(f"\n=== EASE RECOMMENDATION FOR USER {user_id} ===")

  for recommendation in recommendations:
    print(f"Game {recommendation}: {get_steam_game_title_from_id(recommendation)}")


=== EASE RECOMMENDATION FOR USER 1296292 ===
Game 976730: Halo: The Master Chief Collection
Game 262060: Darkest Dungeon®
Game 1286350: BPM: BULLETS PER MINUTE
Game 505460: Foxhole
Game 1612780: The Centennial Case : A Shijima Story
Game 779340: Total War: THREE KINGDOMS
Game 678950: DRAGON BALL FighterZ
Game 359320: Elite Dangerous
Game 266510: Hand of Fate
Game 1286680: Tiny Tina's Wonderlands

=== EASE RECOMMENDATION FOR USER 7660555 ===
Game 50300: Spec Ops: The Line
Game 383870: Firewatch
Game 1874490: Potionomics
Game 212780: New Star Soccer 5
Game 440650: Niche - a genetics survival game
Game 417880: Everybody's Gone to the Rapture
Game 239030: Papers, Please
Game 481510: Night in the Woods
Game 290340: Armello
Game 703080: Planet Zoo

=== EASE RECOMMENDATION FOR USER 13215940 ===
Game 220: Half-Life 2
Game 620: Portal 2
Game 550: Left 4 Dead 2
Game 50: Half-Life: Opposing Force
Game 377160: Fallout 4
Game 367520: Hollow Knight
Game 70: Half-Life
Game 380: Half-Life 2: Episode 